In [1]:
#import tuxml
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

#df = tuxml.load_dataset()
# path = r"x264/sampledConfigurations_distBased_t3.csv"
#path = r"sampledConfigurations_distBased_t3.csv"
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']

# for config_name in samples_config:
#     path = r"sampledConfigurations_"+config_name+"_t3.csv"
# #     df.append(pd.read_csv(path, sep=';'))
#     display(path)

# df = pd.read_csv(path, sep=';')


In [2]:
# df_importance = pd.read_csv("feature_importanceRF.csv")
# df_importance = df[df_importance[:100]["Unnamed: 0"].values]
# df_importance.head()

In [2]:
#size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
#              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
#?              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

size_methods = ["Performance"]
#additional_ft = ["nbyes", "nbno", "nbmodule", "nbyesmodule"]

In [3]:
def features_engine(df):
    
    engine_values = []
    for i, j in df.iterrows():
        num1s = 0
        num1s = j.value_counts()[1]
        engine_values.append(num1s)
    
    df['construction'] = engine_values
    

In [4]:
path = r"sampledConfigurations_distBased_t3.csv"
df = pd.read_csv(path, sep=';')
features_engine(df)
display(df)

,root,filterOff,HeaderCompressionOff,mtOff,CompressionMethod,LZMA,LZMA2,PPMd,BZip2,Deflate,...,BlockSize_4096,x,x_0,x_2,x_4,x_6,x_8,x_10,Performance,construction
0,1,0,0,0,1,0,1,0,0,0,...,0,1,1,0,0,0,0,0,3320,9
1,1,1,1,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,1710,12
2,1,1,1,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,1610,12
3,1,1,1,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,1620,12
4,1,1,1,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,1610,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,1,1,1,1,1,0,0,0,0,1,...,0,1,0,0,0,0,0,1,46400,13
4087,1,1,1,1,1,0,0,0,0,1,...,0,1,0,0,0,0,0,1,44660,13
4088,1,1,1,1,1,0,0,0,0,1,...,0,1,0,0,0,0,0,1,46140,13
4089,1,1,1,1,1,0,0,0,0,1,...,0,1,0,0,0,0,0,1,45120,13


In [5]:
#learning a model by using linear regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

def run_regressorML(reg, test_size, size_target, x_train, x_test):
    assert(size_target in size_methods)
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    #y_pred = reg.intercept_ + np.sum(reg.coef_ * x_test.values, axis=1)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    return dfErrors["% error"].describe()

In [6]:
def ft_importances(coef_, col):
    importanceSeries = pd.Series(coef_, index=col.values)
    return importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)

In [7]:
df_result  = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'test_size', 'coef_order', 'accuracy'])
#df_result  = pd.DataFrame(columns = ['algorithm_name', 'hyperparameters', 'size_target', 'additional_ft', 'test_size', 'coef_order', 'accuracy'])

In [8]:
df_result

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn import metrics

ft_importance_enable = True
# def runFeatureSelection(df, config_name, num_features): 
for config_name in samples_config:
    for samplePerc in [0.3,0.4,0.5,0.6]:
        for t in [1,2,3]:
            
            path_name = config_name+"_t"+str(t)
            path = r"sampledConfigurations_"+path_name+".csv"
            
            df = pd.read_csv(path, sep=';')
            num_features = round(samplePerc*len(df.columns))

            df_importance = pd.read_csv("feature_importance_"+path_name+".csv")
            df_importance = df[df_importance[:num_features]["Unnamed: 0"].values]
            
            print("DF with Feature Importance")
            print("Sample features number: "+ str(num_features))
            display(df_importance)
            df_importance.head()

            target = "Performance"
            paramsLasso = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                           'max_iter': [1, 5, 10, 100, 1000],}
            paramsRidge = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
            paramsENet = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                          'l1_ratio': [0.0, 0.5, 1.0, 0.1],
                          'max_iter': [1, 5, 10, 100, 1000],}

            for ft_dummyfication in [False, True]:
                if ft_dummyfication:
                    #Creating dummy variables in pandas
                    df = pd.get_dummies(df, columns=list(df.drop(columns=size_methods).columns.values))
#                     df_importance = pd.get_dummies(df_importance, columns=list(df_importance.drop(columns=size_methods).columns.values))
                    df_importance = pd.get_dummies(df_importance, columns=list(df_importance.columns.values))

                if ft_importance_enable:
                    ft_selection =  {'None': df.drop(columns=size_methods).columns,
                                'RandomForest': df_importance.columns[:num_features],}
                    
                else:
                    ft_selection =  {'None': df.drop(columns=size_methods).columns,
                                     'RandomForest': df.columns[:num_features],}

                for origin_ft_selection in ft_selection:
                    for size in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:

                        x_train, x_test, y_train, y_test = train_test_split(df[ft_selection[origin_ft_selection]], df[target], test_size=size, random_state=42)  
                        x_train, y_train = np.array(x_train.values), np.array(y_train.values)
                        
                        print(x_train)
                        print(y_train)
                        
                        if len(x_train) > 3:

                            model = {'LinearRegression': LinearRegression(),
                                     'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=3).fit(x_train, y_train).best_estimator_,
                                     'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=3).fit(x_train, y_train).best_estimator_,
                                     'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=3).fit(x_train, y_train).best_estimator_,
                                     'PolynomialRegression': LinearRegression()}
                        else:
                            if len(x_train) >= 2:
                                model = {'LinearRegression': LinearRegression(),
                                         'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=2).fit(x_train, y_train).best_estimator_,
                                         'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=2).fit(x_train, y_train).best_estimator_,
                                         'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=2).fit(x_train, y_train).best_estimator_,
                                         'PolynomialRegression': LinearRegression()}
                        for key in model:
                            if (key == 'PolynomialRegression') and (origin_ft_selection == 'None'):
                                print("It doesn't scale")
                            else:
                                if (key == 'PolynomialRegression'):
                                    #for 2 options we have: 𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²
                                    x_train = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_train)
                                    x_test = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_test)

                                res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train, x_test=x_test)

                                if (key == 'PolynomialRegression'):
                                    model[key].coef_ = model[key].coef_[:num_features]

#                                 print(model[key].coef_)
                                coef_order = ft_importances(model[key].coef_, col=ft_selection[origin_ft_selection])
                                df_result.loc[len(df_result)] = [key, ft_dummyfication, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, size, coef_order, res_model]
            display(df_result)
            if ft_importance_enable:
                df_result.to_csv("results_with_feature_importance_"+path_name+"_"+str(num_features)+".csv", header=True)
            else:
                df_result.to_csv("results_"+path_name+"_"+str(num_features)+".csv", header=True)

DF with Feature Importance
Sample features number: 14


,x_8,BZip2,x_6,PPMd,LZMA2,LZMA,Deflate,Files_20,tmOff,x_0,BlockSize_16,x_4,HeaderCompressionOff,BlockSize_4
0,0,0,0,1,0,0,0,0,1,0,0,1,1,0
1,0,0,0,1,0,0,0,0,1,0,0,1,1,0
2,0,0,0,0,1,0,0,0,0,1,1,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,0,0,0,0,1,0
5,0,0,0,0,1,0,0,0,0,0,0,1,1,0
6,0,0,0,0,1,0,0,0,0,0,0,1,1,0
7,0,0,0,0,1,0,0,0,1,1,0,0,0,0
8,0,0,0,0,1,0,0,0,1,1,0,0,0,0
9,0,0,0,0,1,0,0,0,1,0,0,1,1,0


[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 1 0]]
[ 3210  6210  3260  6080  2490 18820  3050  3330  2950  3080  3050  3370
  2850  1610  6060  3220  3140  1630 20520  3060  3990  7020  1950  3260
  2040  2490  1740  3430  1920  2960  3010  3250  3070  6980 21270]
It doesn't scale
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 1 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 1 0]]
[ 2490 18820  3050  3330  2950  3080  3050  3370  2850  1610  6060  3220
  3140  1630 20520  3060  3990  7020  1950  3260  2040  2490  1740  3430
  1920  2960  3010  3250  3070  6980 21270]
It doesn't scale
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 1 0]]
[ 2950  3080  3050  3370  2850  1610  6060  3220  3140  1630 20520  3060
  3990  7020  1950  3260  2040  2490  1740  3430  1920  2960  3010  3250
  3070  6980 21270]
It doesn't scale
[[1 1 1 ... 0 0 0]


[[0 0 0 0 0 1 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0 1 0 0 1 1 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 1 1]
 [0 0 0 1 0 0 0 0 1 0 0 1 1 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 0 0 1 1 0]
 [1 0 0 0 0 0 1 1 1 0 0 0 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 1 0]
 [0 0 0 0 1 0 0 0 0 1 1 0 0 0]
 [0 1 0 0 0 0 0 0 1 0 0 1 1 0]
 [0 0 0 0 0 0 1 0 1 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 1 1 0 0 1 1 1]
 [0 0 0 1 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 1 1 0]
 [0 0 0 1 0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 1 0]
 [0 0 0 0 1 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 1 0 0 1 1 0]
 [1 0 0 0 0 0 1 0 1 0 0 0 1 1]]
[ 2950  3080  3050  3370  2850  1610  6060  3220  3140  1630 20520  3060
  3990  7020  1950  3260  2040  2490  1740  3430  1920  2960  3010  3250
  3070  6980 212

It doesn't scale
[[1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1
  0 1 1 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1]
 [1 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 0 1 1 1 0 1 0
  1 1 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0 1]
 [1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 1
  0 1 1 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 0 1 0 0 1 1]]
[ 3070  6980 21270]
It doesn't scale
[[1 0 1 0 1 0 1 0 1 0 1 0 0 1]
 [1 0 0 1 1 0 1 0 1 0 1 0 1 0]
 [1 0 1 0 1 0 1 0 0 1 1 0 1 0]
 [1 0 0 1 1 0 1 0 1 0 1 0 1 0]
 [1 0 1 0 1 0 0 1 1 0 1 0 1 0]
 [0 1 1 0 1 0 1 0 1 0 0 1 1 0]
 [1 0 1 0 1 0 1 0 1 0 0 1 1 0]
 [1 0 1 0 1 0 1 0 0 1 1 0 1 0]
 [1 0 1 0 1 0 1 0 1 0 0 1 1 0]
 [1 0 1 0 1 0 1 0 1 0 0 1 1 0]
 [1 0 1 0 1 0 1 0 1 0 0 1 1 0]
 [1 0 1 0 1 0 1 0 0 1 1 0 1 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 0 1]
 [1 0 1 0 1 0 0 1 1 0 1 0 1 0]
 [1 0 0 1 1 0 1 0 1 0 1 0 1 0]
 [1 0 1 0 1 0 1 0 0 1 1 0 1 0]
 [1 0 1 

...

,BZip2,x_6,x_10,PPMd,x_0,Deflate,LZMA,Files_0,x_4,LZMA2,...,Files_50,Files_30,Files_10,BlockSize_16,BlockSize_1,Files_60,BlockSize_128,Files_80,Files_70,BlockSize_32
0,0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
596,0,0,0,0,1,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
597,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
598,0,0,0,0,1,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


[[1 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 1 0 0]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]]
[ 3400  3360  2060  1730  1830  3350  1860  2230  3550  2680  1970  3220
  6780  2910  5400  1760  2020  3780  1800  5430  2720  7130  2570  2540
  3410  3420  2860  3240  2990  3000  3770  2010  4030  1880  2080  2660
  3190  3390  5430  3310  3190  2980  2400  1940  3470  6380  2900  2670
  1970  6690  2930  6120  1680  1900  2870  6300  1940 19820  1950  6050
  3520  3370  6480  2090  2960  3060  1670  2000  1890  6070  3160  2000
  3310  3520  5560  2910  3580  3210  2020  3670  3600  4070  3120  3220
  2130  3440  6010  6500  3940  3320  2180  1720  2510  2120  1680  3030
  4080  6490  3380  3230  1900  2940  3660  5570  2660  2960  2720  2140
  2330  2690  3690  2780  2010  6500  3810  3240  1660  3410  3210  1960
  3540  2020  2460  6770  6220  6250  6790  6090  6050  2060  6640  1940
  3450  2980  6240  1850  6260  3000  2120  3240  2950  5440  6280  3370
  30

It doesn't scale
[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 1 0 0]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]]
[ 1760  6280  5430  3680  2920  2030  2200  6150  3400  3990  3450  4100
  3600  1750  3790  3230  2780  3090  4200  1960  3840  2020  1760  2920
  2710  1830  7900  3190  2930  2010  2320  2030  3310  3310  6020  1760
  5420  3930  3370  1730  2480  3220  2150  6190  2680  5590  2060  3230
  4010  7070  2020  3230  2950  3940  2900  3660  1750  3160  1760  2760
  2960  1800  1800  1730  3360  2430  3690  2000  2960  3010  3010  2360
  2680  2010  1980  4010  6310  3580  6700  1710  3360  6220  5470  6780
  3090  1900  2530  2000  6210  3560  6250  3630  3040  2880  1990  2270
  4080  3460  2340  2610  2000  1720  1980  3040  2050  2220  1740  3480
  5400  1870  2930  2620 19730  5420  2180 19160  6000  2690  2720  1720
  3080  1780  3880  3290  2920  6920  3840  3000  2810  2000  6460  3140
  3960  3820  2850  3380  1740  5890  6020  1780  1750  6200

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[ 3520  3370  6480  2090  2960  3060  1670  2000  1890  6070  3160  2000
  3310  3520  5560  2910  3580  3210  2020  3670  3600  4070  3120  3220
  2130  3440  6010  6500  3940  3320  2180  1720  2510  2120  1680  3030
  4080  6490  3380  3230  1900  2940  3660  5570  2660  2960  2720  2140
  2330  2690  3690  2780  2010  6500  3810  3240  1660  3410  3210  1960
  3540  2020  2460  6770  6220  6250  6790  6090  6050  2060  6640  1940
  3450  2980  6240  1850  6260  3000  2120  3240  2950  5440  6280  3370
  3010  2740  3890  2670  3580  6060  3350  2250  3380  3720  3360  2910
  2000  3440  2240  3590  3690  6010  3220  2110  2060  6780  2720  2010
  3290  2710  4000  6180  3060  7430  1730  3530  6570  1980  5530  2180
  1760  6280  5430  3680  2920  2030  2200  6150  3400  3990  3450  4100
  3600  1750  3790  3230  2780  3090  4200  1960  3840  2020  1760  2920
  27

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[3080 1780 3880 3290 2920 6920 3840 3000 2810 2000 6460 3140 3960 3820
 2850 3380 1740 5890 6020 1780 1750 6200 2900 1690 2970 1950 3250 3150
 2910 1810 6220 6070 2190 6240 1750 7330 6370 1960 2000 1930 6500 6890
 3490 2940 6660 5450 6420 2920 1880 2920 2280 3740 2000 2140 3580 2680
 2880 5440 2940 1750 2900 2990 3340 3370 2810 6440 6250 3060 1750 5710
 2690 3210 2090 2570 1900 3730 3800 1740 6030 3000 1770 3450 3880 3530
 2700 3250 3320 3640 2510 3020 3380 3580 3800 3670 2960 3670 6280 1660
 3180 1800 4070 3240 2880 2210 2790 1880 3510 6540 1730 3310 1750 2290
 3930 6050 2820 2780 6020 2960 3140 3290 3520 1740 1800 5460 3870 3210
 6010 2500 1740 3040 2790 3810 3710 2700 2980 4160 5510 3440 4060 6610
 1870 2710 3630 2900 3970 1760 1750 3280 2520 1980 6240 2780 6840 6010
 1730 3330 3850 5400 2940 2110 2080 4090 3680 3030 3800 2430 6250 3040
 3020 3310 1990 2000 3690 61

In [ ]:
len(x_train)

In [ ]:
df_result['accuracy'][0]

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

rs = []
for i in range(len(df_result['accuracy'])):
    rs.append(df_result['accuracy'][i]['mean'])

# rs
# display (df_result['algorithm_name'], df_result['accuracy'], rs)

plt.scatter(df_result['algorithm_name'], rs, color ="blue", marker = "s")
plt.plot(df_result['algorithm_name'], rs, color ="blue")


In [ ]:
# Boxplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


data = [model['LinearRegression'].coef_, model['Lasso'].coef_, model['Ridge'].coef_, model['ElasticNet'].coef_, model['PolynomialRegression'].coef_]
fig1, ax1 = plt.subplots()
ax1.set_title('Coeficient of feature importance for all algorithms')
ax1.boxplot(data)
plt.show()